In [29]:
import openai
import instructor
from pydantic import BaseModel, Field
from qdrant_client import QdrantClient


### Mock Example

In [3]:
prompt = """
 You are a helpful assistant. 
 Return an answer to the question. 
 Question: What is you name? 
"""

In [8]:
response = openai.chat.completions.create(
 model="gpt-4.1-mini",
 messages=[
    { "role":"system", "content":"prompt"}
 ]
)
print(response.choices[0].message.content)

Hello! How can I assist you today?


### Add instructor (structured outputs)

In [22]:
client = instructor.from_openai(openai.OpenAI())


In [23]:
class RAGGenerationResponse(BaseModel):
    answer:str = Field(description="Answer to the question.")



In [25]:

response, raw_response = client.chat.completions.create_with_completion(
 model="gpt-4.1-mini",
 messages=[
    { "role":"system", "content":"prompt"}
 ],
 response_model=RAGGenerationResponse
)
# print(response.choices[0].message.content)

In [27]:
response.model_dump()

{'answer': 'Based on the provided prompt, the user has requested you to think step-by-step and carefully analyze the problem, and then solve it. However, they have not actually stated a specific problem or question to solve. To assist you efficiently, please provide the problem or question you want me to solve.'}

In [28]:
raw_response

ChatCompletion(id='chatcmpl-CUpEpPR4yXLn6uzrE1nkldFu1ve11', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_1vunRNnBNilY4zqkcBhJycvH', function=Function(arguments='{"answer":"Based on the provided prompt, the user has requested you to think step-by-step and carefully analyze the problem, and then solve it. However, they have not actually stated a specific problem or question to solve. To assist you efficiently, please provide the problem or question you want me to solve."}', name='RAGGenerationResponse'), type='function')]))], created=1761462835, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_4c2851f862', usage=CompletionUsage(completion_tokens=63, prompt_tokens=75, total_tokens=138, completion_tokens_details=CompletionTokensDetails(a

### RAG Example

In [31]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    retrieved_context_ratings = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"],context["retrieved_context_ratings"]):
        formatted_context += f"- {id}:, rating:{rating}, description:{chunk}\n"
      

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

def generate_answer(prompt):
    client = instructor.from_openai(openai.OpenAI())

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
        response_model=RAGGenerationResponse
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "answer": answer.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [30]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [32]:
output = rag_pipeline("Can I get a tablet",qdrant_client)

In [33]:
output

{'answer': 'Yes, you can get a tablet. There are several tablets available including:\n- DUODUOGO 2 in 1 Tablet 10 Inch with Keyboard Mouse Stylus, 128GB Expandable, 64GB ROM, 4GB RAM, Android 10.0, Quad-Core HD IPS Screen\n- DOOGEE T10 Android Tablet 2023, 10.1" FHD+ Android 12 Tablet, 15GB RAM + 128GB Storage, Octa-Core, 8300mAh Battery\nThese tablets offer various features suitable for study, entertainment, gaming, and video conferencing.',
 'question': 'Can I get a tablet',
 'retrieved_context_ids': ['B0B44TGKRX',
  'B0C35RS6MS',
  'B0BK8458S2',
  'B0B23RZWTH',
  'B0C2C7NMGP'],
 'retrieved_context': ['DUODUOGO 2 in 1 Tablet 10 Inch with Keyboard Mouse Stylus, 128GB Expand 64GB ROM 4GB RAM Android 10.0 Quad-Core HD IPS Screen 8MP Dual Camera GPS FM OTG Bluetooth 4G Dual SIM & WiFi, DGO-P610.1 Inch HD IPS Tablet - This tablet features a 1280*800 HD G+G IPS touch display, wide viewing angle, bright display, more vivid colors and top-notch picture quality. High Performance 2 in 1 Table